In [1]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install sklearn
!pip install librosa
!pip install IPython
!pip install warnings
!pip install os


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for warnings
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

df3 = pd.read_csv("lib/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv")
df30 = pd.read_csv("lib/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv")

In [7]:
import os
import time

df_base = df3
data_numeric = df_base.drop(columns=['filename', 'length'])
genre_means = data_numeric.groupby('label').mean()
global_std = data_numeric.std(numeric_only=True)
genres = genre_means.index.tolist()

files_test_limit = 5

In [3]:
import librosa

def extract_features_from_buffer(y, sr):
    f = {}
    # Podstawowe cechy (takie same jak w Twoim CSV)
    f['chroma_stft_mean'] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    f['chroma_stft_var'] = np.var(librosa.feature.chroma_stft(y=y, sr=sr))
    f['rms_mean'] = np.mean(librosa.feature.rms(y=y))
    f['rms_var'] = np.var(librosa.feature.rms(y=y))
    f['spectral_centroid_mean'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    f['spectral_centroid_var'] = np.var(librosa.feature.spectral_centroid(y=y, sr=sr))
    f['spectral_bandwidth_mean'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    f['spectral_bandwidth_var'] = np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    f['rolloff_mean'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    f['rolloff_var'] = np.var(librosa.feature.spectral_rolloff(y=y, sr=sr))
    f['zero_crossing_rate_mean'] = np.mean(librosa.feature.zero_crossing_rate(y))
    f['zero_crossing_rate_var'] = np.var(librosa.feature.zero_crossing_rate(y))

    y_harm, y_perc = librosa.effects.hpss(y)
    f['harmony_mean'], f['harmony_var'] = np.mean(y_harm), np.var(y_harm)
    f['perceptr_mean'], f['perceptr_var'] = np.mean(y_perc), np.var(y_perc)

    # Ważne: obsługa tempa (może zwracać tablicę)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    f['tempo'] = tempo[0] if isinstance(tempo, (np.ndarray, list)) else tempo

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for i in range(1, 21):
        f[f'mfcc{i}_mean'] = np.mean(mfccs[i-1])
        f[f'mfcc{i}_var'] = np.var(mfccs[i-1])

    return pd.Series(f)

In [4]:
def get_signature_features(target_genre, top_n=8):
    """
    Dla danego gatunku sprawdza starcia 1-na-1 ze wszystkimi innymi.
    Wybiera cechy, które najczęściej okazywały się najbardziej skrajne.
    """
    feature_wins = {col: 0 for col in genre_means.columns}

    for other_genre in genres:
        if target_genre == other_genre: continue

        # Różnica średnich znormalizowana przez odchylenie standardowe
        diff = (genre_means.loc[target_genre] - genre_means.loc[other_genre]).abs() / global_std

        # Bierzemy 3 cechy, które najbardziej dzielą te dwa konkretne gatunki
        top_diffs = diff.sort_values(ascending=False).head(3).index
        for feat in top_diffs:
            feature_wins[feat] += 1

    return pd.Series(feature_wins).sort_values(ascending=False).head(top_n).index.tolist()

In [6]:
def classify_with_voting(file_path):
    # Wczytujemy tylko pierwsze 10 sekund, żeby oszczędzić czas
    y_full, sr = librosa.load(file_path, duration=10)
    segment_len = 3 * sr

    votes = []

    # Analizujemy 3 fragmenty po 3 sekundy
    for i in range(3):
        start = i * segment_len
        end = start + segment_len
        if end > len(y_full): break

        y_seg = y_full[start:end]
        f_seg = extract_features_from_buffer(y_seg, sr)

        errors = {}
        for g in genres:
            # Twoja logika starć 1-na-1
            best_feats = get_signature_features(g)
            g_vals = genre_means.loc[g, best_feats] / global_std[best_feats]
            f_vals = f_seg[best_feats] / global_std[best_feats]
            errors[g] = np.mean(np.abs(g_vals - f_vals))

        votes.append(min(errors, key=errors.get))

    # Zwraca najczęstszy wynik
    return max(set(votes), key=votes.count)

In [8]:
base_dir = 'lib/gtzan-dataset-music-genre-classification/Data/genres_original'
results = []
start_time = time.time()

for folder in genres:
    folder_path = os.path.join(base_dir, folder)
    if not os.path.exists(folder_path): continue

    # Pobierz listę plików i ogranicz ją
    files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    files_to_test = files[:files_test_limit]

    print(f"Testuję {folder.upper()}...")

    for file in files_to_test:
        full_path = os.path.join(folder_path, file)
        try:
            pred = classify_with_voting(full_path)
            results.append({'Real': folder, 'Pred': pred, 'Correct': folder == pred})
        except Exception as e:
            print(f"Błąd w {file}: {e}")

end_time = time.time()
print(f"\n--- TEST ZAKOŃCZONY w {int(end_time - start_time)} sekund ---")

# Raport
report = pd.DataFrame(results)
print(f"Skuteczność: {report['Correct'].mean()*100:.2f}%")

Testuję BLUES...
Testuję CLASSICAL...
Testuję COUNTRY...
Testuję DISCO...
Testuję HIPHOP...
Testuję JAZZ...
Testuję METAL...
Testuję POP...
Testuję REGGAE...
Testuję ROCK...

--- TEST ZAKOŃCZONY w 27 sekund ---
Skuteczność: 32.00%
